# Hazard check

In [ ]:
from climada.hazard import Hazard, Centroids, TCTracks, TropCyclone
from climada.entity import Exposures
from climada.entity.exposures import LitPop
# from climada.entity import Measure, MeasureSet, Entity
from climada.engine import ImpactCalc, Impact
import climada.hazard.tc_tracks as tc_tracks
import climada.hazard.tc_tracks_synth as tc_tracks_synth
from pathos.pools import ProcessPool
# import os
# from climada.util import save, load
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import date2num
import pandas as pd
import math
import collections
import climada.util.coordinates as u_coord
# import climada.entity.exposures.litpop as lp
import geopandas as gpd

In [ ]:
## set parameters
# Exposure
my_country = 'AUS'
m, n = 1,1 # based on analysis by Eberenz et al. (2020), (1,1) is the best combination.
my_fin_mode = 'pc'
ref_year = 2023

my_res_arcsec = 600
sid = '1995348S15135' # '1994344S09112'

## Exposure
exp = LitPop.from_countries(my_country, my_res_arcsec, (m,n), my_fin_mode, reference_year = ref_year)
exp.check()

## Centroids
lat = exp.gdf.latitude
lon = exp.gdf.longitude
polygon = (lon.min(), lon.max(), lat.min(), lat.max())

cent = Centroids.from_pnt_bounds((lon.min(), lat.min(), lon.max(), lat.max()), res=my_res_arcsec/3600)
cent.set_on_land()
cent.set_region_id()
# cent.plot(alpha=0.1);

In [ ]:
pool = ProcessPool(nodes=8)

tc1 = TCTracks.from_ibtracs_netcdf(storm_id=sid)
tc1.equal_timestep(land_params = True, pool = pool)

haz = TropCyclone.from_tracks(tc1, centroids = cent, pool = pool)
haz.check()

In [ ]:
# Generate synthetic tracks: with decay
np.random.seed(1)
pool = ProcessPool(nodes=8)

tc_decay = TCTracks.from_ibtracs_netcdf(storm_id=sid)
tc_decay.equal_timestep(land_params = True, pool=pool)
tc_decay.calc_perturbed_trajectories(nb_synth_tracks = 10, decay=True, pool=pool);

haz_decay = TropCyclone.from_tracks(tc_decay, centroids = cent, pool = pool)
haz_decay.check()

In [ ]:
# Generate synthetic tracks: without decay
np.random.seed(1)
pool = ProcessPool(nodes=8)

tc_no_decay = TCTracks.from_ibtracs_netcdf(storm_id=sid)
tc_no_decay.equal_timestep(land_params = True, pool=pool)
tc_no_decay.calc_perturbed_trajectories(nb_synth_tracks = 10, decay=False, pool=pool);

haz_no_decay = TropCyclone.from_tracks(tc_no_decay, centroids = cent, pool = pool)
haz_no_decay.check()

pool.close()
pool.join()

In [ ]:
tc1.plot(figsize=(9,5));
tc_decay.plot(legend = False, figsize=(9,5));
tc_no_decay.plot(legend=False, figsize=(9,5));

In [ ]:
# https://github.com/CLIMADA-project/climada_python/issues/151

def get_lf_df(tr_df):
    lf_start = tr_df.iloc[np.where(np.diff(tr_df['on_land'].astype(float)) == 1)[0],:]['time']
    lf_end = tr_df.iloc[np.where(np.diff(tr_df['on_land'].astype(float)) == -1)[0],:]['time']
    if len(lf_end) < len(lf_start):
        lf_end = lf_end.append(max(tr_df['time']))
    lf_dat = pd.DataFrame({'start_lf':lf_start, 'end_lf': lf_end}).reset_index(drop=True)
    return lf_dat


def get_tracks_df(tracks, sid_hist, ens_nb):
    # get historical track df
    tr_hist = TCTracks()
    tr_hist.data = [tracks.get_track(sid_hist)]
    tr_hist_df = tr_hist.to_geodataframe(as_points=True)
    # get synthetic track df
    tr_synth = TCTracks()
    tr_synth.data = [tracks.get_track(sid_hist+'_gen'+str(ens_nb))]
    tr_synth_df = tr_synth.to_geodataframe(as_points=True)
    return tr_synth_df, tr_hist_df

def plot_landfall_data(tracks, sid_hist, ens_nb):
    tr_df, tr_hist_df = get_tracks_df(tracks, sid_hist, ens_nb)
    fig, ax = plt.subplots(figsize=(10,6))
    lf_dat = get_lf_df(tr_df)
    lf_hist = get_lf_df(tr_hist_df)
    #     ws = ax.scatter(tr_df['time'],tr_df['max_sustained_wind'])
    ws = ax.plot(tr_df['time'],tr_df['max_sustained_wind'], marker='x')
    wo = ax.scatter(tr_hist_df['time'], tr_hist_df['max_sustained_wind'], marker='+', color='red')
    # ax = pd.DataFrame(t1_syn_df).plot(x='time',y='max_sustained_wind')
    for i in range(lf_hist.shape[0]):
        lfo = ax.axvspan(date2num(lf_hist['start_lf'][i]), date2num(lf_hist['end_lf'][i]), 
                   label="Landfalls",color="grey", alpha=0.3)
    for i in range(lf_dat.shape[0]):
        lfs = ax.axvspan(date2num(lf_dat['start_lf'][i]), date2num(lf_dat['end_lf'][i]), 
                   label="Landfalls",color="green", alpha=0.3)
    ax.set_title('Maximum sustained wind for cyc_id %s' % sid_hist+'_gen'+str(ens_nb))
    ax.set_ylabel('kts')
    legend1 = plt.legend((ws[0], wo), ('Actual', 'Historical track'), loc=2)
    ax.legend((lfs, lfo), ('Landfall', 'Landfall (historical)'), loc=3)
    plt.gca().add_artist(legend1)

    fig, ax = plt.subplots(figsize=(10,6))
    ws = ax.plot(tr_df['time'],tr_df['central_pressure'], marker='x')
    wo = ax.scatter(tr_hist_df['time'], tr_hist_df['central_pressure'], marker='+', color='red')
    we = ax.scatter(tr_hist_df['time'], tr_hist_df['environmental_pressure'], marker='.', color='orange')
    #     wes = ax.scatter(tr_df['time'], tr_df['environmental_pressure'], marker='.', color='green')
    # ax = pd.DataFrame(t1_syn_df).plot(x='time',y='max_sustained_wind')
    for i in range(lf_hist.shape[0]):
        ax.axvspan(date2num(lf_hist['start_lf'][i]), date2num(lf_hist['end_lf'][i]), 
                   label="Landfalls",color="grey", alpha=0.3)
    for i in range(lf_dat.shape[0]):
        ax.axvspan(date2num(lf_dat['start_lf'][i]), date2num(lf_dat['end_lf'][i]), 
                   label="Landfalls",color="green", alpha=0.3)
    ax.set_title('Central pressure for cyc_id %s' % sid_hist+'_gen'+str(ens_nb))
    ax.set_ylabel('hPa')
    ax.legend((ws[0], wo, we), ('Actual', 'Historical track', 'Environmental pressure'))


In [ ]:
plot_landfall_data(tc_decay, sid, 1)

In [ ]:
haz_decay.centroids.gdf

In [ ]:
haz.plot_intensity(event=1);
haz_decay.plot_intensity(event=2);
haz_no_decay.plot_intensity(event=2);

In [ ]:
tc_tracks._get_landfall_idx(tc_no_decay.data[1])

In [ ]:
tc_idx = 138
print('TCTracks decay:', tc_decay.data[1].lat[tc_idx].values,
	  tc_decay.data[1].lon[tc_idx].values,
	  tc_decay.data[1].max_sustained_wind[tc_idx].values,
	  tc_decay.data[1].max_sustained_wind_unit,
	  tc_decay.data[1].on_land[tc_idx].values)

print('TCTracks no decay:', tc_no_decay.data[1].lat[tc_idx].values,
	  tc_no_decay.data[1].lon[tc_idx].values,
	  tc_no_decay.data[1].max_sustained_wind[tc_idx].values,
	  tc_no_decay.data[1].max_sustained_wind_unit,
	  tc_no_decay.data[1].on_land[tc_idx].values)

haz_idx = 22369
print('Decay:', haz_decay.centroids.lat[haz_idx],
	  haz_decay.centroids.lon[haz_idx],
	  haz_decay.intensity[1].toarray().flatten()[haz_idx],
	  haz_decay.units)

print('No decay:', haz_no_decay.centroids.lat[haz_idx],
	  haz_no_decay.centroids.lon[haz_idx],
	  haz_no_decay.intensity[1].toarray().flatten()[haz_idx],
	  haz_no_decay.units)



Note 1: decay function does work, but there is nothing in the function that stops the TC early if the intensity reaches below a certain threshold.

We need to look at the function that turns TCTracks object into TropCyclone object.